In [25]:
import pandas as pd
import numpy as np
import nltk
# Cleaning the texts
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import RandomOverSampler
from joblib import dump, load
import pickle

In [26]:
data = pd.read_excel('1000 leads.xlsx',encoding = 'ISO-8859-1')

In [27]:
data.head()

,Lead Name,Location,Status,Status information,Unnamed: 4
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn...",NaN
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema...",NaN
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr,NaN
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...,NaN
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ...",NaN


In [28]:
data.drop(['Lead Name', 'Unnamed: 4'],axis =1,inplace=True)

In [29]:
data.isnull().sum()

Location              21
Status                 3
Status information    24
dtype: int64

In [30]:
data_new=data.dropna()

In [31]:
data_new.isnull().sum()

Location              0
Status                0
Status information    0
dtype: int64

In [32]:
data_new.reset_index(drop=True, inplace=True)

In [33]:
data_new = data_new.rename(columns={'Status information': 'Status_information'})
data_new = data_new.rename(columns={'Status ': 'Status'})
data_new.columns

Index(['Location', 'Status', 'Status_information'], dtype='object')

In [34]:
# to remove functuation

data_new.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [35]:
import string
data_lower=data_new.apply(lambda x: x.astype('str').str.lower())
data_lower.head()

,Location,Status,Status_information
0,hyderabad,not converted,prema share me details available in evn...
1,pune,not converted,prema cal me tmrw shared details to ema...
2,bangalore,converted,aug mohan rnr
3,mumbai,not converted,surendra i want only server aug mo...
4,hyderabad,not converted,prema rnr prema gave info he said ...


In [36]:
# removes unnecessary spaces
data_lower.apply(lambda x: x.astype(str).str.strip())

,Location,Status,Status_information
0,hyderabad,not converted,prema share me details available in evng ...
1,pune,not converted,prema cal me tmrw shared details to email ...
2,bangalore,converted,aug mohan rnr
3,mumbai,not converted,surendra i want only server aug mohan cal ...
4,hyderabad,not converted,prema rnr prema gave info he said he wi...
...,...,...,...
951,bangalore,not converted,surendra please send me details tomorrow afte...
952,chennai,not converted,prema need ct but our venue is far shared ...
953,bangalore,not converted,apr mohan intrstd in evng batch trail perd ...
954,bangalore,not converted,surendra please share me details gowt...


In [37]:
data_lower['Status'].unique()

array(['not converted', 'converted ', 'conveted'], dtype=object)

In [38]:
data_lower['Status']= data_lower['Status'].replace(['conveted'],'converted ')
data_lower['Status'].unique()

array(['not converted', 'converted '], dtype=object)

In [39]:
data_x=data_lower.drop(['Status'],axis =1)
data_x.head()

,Location,Status_information
0,hyderabad,prema share me details available in evn...
1,pune,prema cal me tmrw shared details to ema...
2,bangalore,aug mohan rnr
3,mumbai,surendra i want only server aug mo...
4,hyderabad,prema rnr prema gave info he said ...


In [40]:
data_lower['Status']=data_lower['Status'].map({'converted ':1,'not converted':0})
Y=data_lower.drop(['Location','Status_information'],axis =1)
Y.head()

,Status
0,0
1,0
2,1
3,0
4,0


In [41]:
Y['Status'].value_counts()

0    829
1    127
Name: Status, dtype: int64

In [42]:
ros = RandomOverSampler(random_state=0)
X_res,Y_res=ros.fit_resample(data_x,Y)

In [43]:
Y_res['Status'].value_counts()

1    829
0    829
Name: Status, dtype: int64

In [44]:
Status_information = []
for row in range(0,len(X_res.index)):
    Status_information.append(' '.join(str(x) for x in X_res.iloc[row,0:2]))

In [45]:
Status_information

['hyderabad      prema  share me details  available in evng      prema  postponed the plans for training currently      prema  not interested now',
 'pune      prema  cal me tmrw  shared details to email      prema  share details to email  will check n revert      prema  received your email  i m looking for asq certification',
 'bangalore    aug mohan rnr',
 'mumbai         surendra  i want only server    aug mohan cal busy    aug mohan reg for server',
 'hyderabad      prema rnr      prema  gave info  he said he will revert in  hr      prema  planning for next month  share details',
 'mumbai         surendra  i will check demo attended demo          surendra  i will get back         surendra  i will think and respond          surendra  not interested',
 'delhi         surendra  asking code',
 'pune    aug mohan mail shred for ml tensorflow corprt    aug mohan shared but rnr',
 'bangalore    aug mohan askng code',
 'mumbai      prema  rnr      prema  share me tableau recorded session c

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(Status_information)

pickle.dump(cv,open('transform_new.pkl','wb'))

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf=TfidfTransformer()

X=tfidf.fit_transform(X)

In [49]:
print(X.shape,Y_res.shape)

(1658, 1246) (1658, 1)


In [50]:
pickle.dump(tfidf,open('transform.pkl','wb'))

In [52]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_res, test_size = 0.2, random_state = 1500)

In [55]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
y_pred = clf.predict(X_test)

In [57]:
expected = y_test

from sklearn import metrics
print(metrics.classification_report(expected, y_pred))
print(metrics.confusion_matrix(expected, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.66      0.75       169
           1       0.72      0.90      0.80       163

    accuracy                           0.78       332
   macro avg       0.80      0.78      0.78       332
weighted avg       0.80      0.78      0.78       332

[[112  57]
 [ 16 147]]


In [58]:
pickle.dump(clf,open('clf.pkl','wb'))

In [59]:
y_pred

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,